# DEA Coastlines generation using command line tools <img align="right" src="https://github.com/GeoscienceAustralia/dea-notebooks/raw/develop/Supplementary_data/dea_logo.jpg">
This notebook demonstrates how to run a DEA Coastlines analysis using command line tools.

### Setup

Set working directory to top level of repo to ensure links work correctly:

In [1]:
cd ..

/home/jovyan/Robbi/dea-coastlines


Update required packages:

In [2]:
pip install -r requirements.in --quiet

You should consider upgrading via the '/env/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


Set analysis parameters:

In [3]:
config_path = 'configs/dea_coastlines_config_development.yaml'
study_area = 8
raster_version = 'development'
vector_version = 'development'
continental_version = 'development'

### Run DEA Coastlines analysis
#### Run tidally-constrained raster generation

In [9]:
!python -m coastlines.raster --help

Usage: python -m coastlines.raster [OPTIONS]

Options:
  --config_path TEXT              Path to the YAML config file defining inputs
                                  to use for this analysis. These are
                                  typically located in the `dea-
                                  coastlines/configs/` directory.  [required]
  --study_area TEXT               A string providing a unique ID of an
                                  analysis gridcell that will be used to run
                                  the analysis. This should match a row in the
                                  "id" column of the provided analysis
                                  gridcell vector file.  [required]
  --raster_version TEXT           A unique string proving a name that will be
                                  used for output raster directories and
                                  files. This can be used to version different
                                  analysis outputs.  [req

Example analysis:

In [8]:
!python -m coastlines.raster --config_path {config_path} --study_area {study_area} --raster_version {raster_version} --start_year 1988 --end_year 2021

#### Run vector annual shoreline and rates of change statistics generation

In [6]:
!python -m coastlines.vector --help

Usage: python -m coastlines.vector [OPTIONS]

Options:
  --config_path TEXT              Path to the YAML config file defining inputs
                                  to use for this analysis. These are
                                  typically located in the `dea-
                                  coastlines/configs/` directory.  [required]
  --study_area TEXT               A string providing a unique ID of an
                                  analysis gridcell that was previously used
                                  to generate raster files. This is used to
                                  identify the raster files that will be used
                                  as inputs for shoreline extraction, and
                                  should match a row in the "id" column of the
                                  provided analysis gridcell vector file.
                                  [required]
  --raster_version TEXT           A unique string providing a name that was
   

Example analysis:

In [9]:
!python -m coastlines.vector --config_path {config_path} --study_area {study_area} --raster_version {raster_version} --start_year 1988 --end_year 2021 --baseline_year 2021

#### Run continental-scale layer generation

In [11]:
!python -m coastlines.continental --help

Usage: python -m coastlines.continental [OPTIONS]

Options:
  --vector_version TEXT           A unique string proving a name that was used
                                  for output vector directories and files.
                                  This is used to identify the tiled annual
                                  shoreline and rates of change layers that
                                  will be combined into continental-scale
                                  layers.  [required]
  --continental_version TEXT      A unique string proving a name that will be
                                  used for output continental-scale layers.
                                  This allows multiple versions of
                                  continental-scale layers to be generated
                                  from the same input vector data, e.g. for
                                  testing different hotspot of coastal change
                                  summary layers. If not

Example analysis:

In [14]:
!python -m coastlines.continental --vector_version {vector_version} --continental_version {continental_version} --shorelines True --ratesofchange True --hotspots True --baseline_year 2021

2023-01-16 00:29:06 INFO Writing data to data/processed/development
Warning 1: A geometry of type MULTILINESTRING is inserted into layer shorelines_annual of geometry type LINESTRING, which is not normally allowed by the GeoPackage specification, but the driver will however do it. To create a conformant GeoPackage, if using ogr2ogr, the -nlt option can be used to override the layer geometry type. This warning will no longer be emitted for this combination of layer and feature geometry type.
2023-01-16 00:29:07 INFO Merging annual shorelines complete
2023-01-16 00:29:09 INFO Merging rates of change points complete
2023-01-16 00:29:10 INFO Loading continental shoreline and rates of change data into memory
2023-01-16 00:29:10 INFO Generating coastal change hotspots
2023-01-16 00:29:10 INFO Calculating 10000 m hotspots
2023-01-16 00:29:10 INFO Calculating 5000 m hotspots
2023-01-16 00:29:11 INFO Calculating 1000 m hotspots
2023-01-16 00:29:12 INFO Writing coastal change hotspots complete
2

## Example combined analysis
This demonstrates how the three components of DEA Coastlines (raster generation, vector generation and continental layers generation) can be applied automatically to a sequence of input study area grid cells.

In [10]:
# Study areas
study_areas = [1, 2, 3, 4, 5, 6, 7, 8]

In [11]:
# Run raster and vector generation for each study area
for study_area in study_areas:
    print(study_area)
    !python -m coastlines.raster --config_path {config_path} --study_area {study_area} --raster_version {raster_version} --start_year 1988 --end_year 2021
    !python -m coastlines.vector --config_path {config_path} --study_area {study_area} --raster_version {raster_version} --vector_version {vector_version} --start_year 1988 --end_year 2021 --baseline_year 2021
    
# When complete, combine into single continental outputs
# !python -m coastlines.continental --vector_version {vector_version} --continental_version {continental_version} --shorelines True --ratesofchange True --hotspots True --baseline_year 2021

1
/env/lib/python3.8/site-packages/distributed/node.py:180: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36699 instead
  warnings.warn(
<Client: 'tcp://127.0.0.1:40571' processes=1 threads=15, memory=117.21 GiB>
2023-01-13 01:05:14 INFO Study area 1: Loaded study area grid
2023-01-13 01:05:21 INFO Study area 1: Loaded virtual product
Creating reduced resolution tide modelling array
Modelling tides using FES2014 tide model
Reprojecting tides into original array
100%|██████████████████████████████████████| 1076/1076 [00:04<00:00, 234.61it/s]
2023-01-13 01:05:49 INFO Study area 1: Finished modelling tide heights
2023-01-13 01:05:49 INFO Study area 1: Calculating low and high tide cutoffs for each pixel
2023-01-13 01:05:49 INFO Study area 1: Started exporting raster data
2023-01-13 01:09:49 INFO Study area 1: Completed exporting raster data
distributed.nanny - WARNING - Worker process still alive after 3.9999990463256

### Clean up and remove files
Warning: this will delete all data processed with a matching unique version name. Only run these cells if you no longer need these processed outputs!

In [ ]:
# rm -rf data/interim/raster/{raster_version}

In [ ]:
# rm -rf data/interim/vector/{vector_version}

In [ ]:
# rm -rf data/processed/{continental_version}

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** For assistance with any of the Python code or Jupyter Notebooks in this repository, please post a [Github issue](https://github.com/GeoscienceAustralia/dea-coastlines/issues/new).

**Last modified:** November 2022

**To cite:**

> Bishop-Taylor, R., Nanson, R., Sagar, S., Lymburner, L. (2021). Mapping Australia's dynamic coastline at mean sea level using three decades of Landsat imagery. Remote Sensing of Environment, 267, 112734. Available: https://doi.org/10.1016/j.rse.2021.112734
>
> Nanson, R., Bishop-Taylor, R., Sagar, S., Lymburner, L., (2022). Geomorphic insights into Australia's coastal change using a national dataset derived from the multi-decadal Landsat archive. Estuarine, Coastal and Shelf Science, 265, p.107712. Available: https://doi.org/10.1016/j.ecss.2021.107712
>
> Bishop-Taylor, R., Sagar, S., Lymburner, L., Alam, I., Sixsmith, J. (2019). Sub-pixel waterline extraction: characterising accuracy and sensitivity to indices and spectra. Remote Sensing, 11 (24):2984. Available: https://doi.org/10.3390/rs11242984